# Counting QPU Access in Qbsolv

In [ ]:
!pip install dwave-ocean-sdk

In [ ]:
import logging
import itertools
import random
import re
from datetime import datetime

import dimod
from dwave.system import DWaveCliqueSampler
from dwave_qbsolv import QBSolv

In [ ]:
logger = logging.getLogger('dwave_qbsolv.qbsolv_binding')
logger.setLevel(logging.DEBUG)

ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch_formatter = logging.Formatter('[%(asctime)s] %(message)s')
ch.setFormatter(ch_formatter)
logger.addHandler(ch)

In [ ]:
def create_problem(qubo_size):
    Q = {t: random.uniform(-1, 1)
         for t in itertools.product(range(qubo_size), repeat=2)}
    return dimod.BinaryQuadraticModel.from_qubo(Q)

In [ ]:
qubo_size = 120
bqm = create_problem(qubo_size)

config = dict(token='YOUR_TOKEN', solver='DW_2000Q_6')
sampler = QBSolv()

In [ ]:
subqubo_size = 30
num_repeats = 1
num_iters = 3

# 実行のたびに、ログを分けて出力
for n in range(num_iters):
    logger.info(f'Iteration: {n}')

    log_file = datetime.now().strftime('%y%m%d%H%M%S%f') + '.log'
    fh = logging.FileHandler(log_file)
    fh.setLevel(logging.DEBUG)
    fh_formatter = logging.Formatter(
        '[%(asctime)s][%(filename)s:%(lineno)d] %(message)s'
    )
    fh.setFormatter(fh_formatter)
    logger.addHandler(fh)

    dw_sampler = DWaveCliqueSampler(**config)
    sampleset = sampler.sample(bqm, num_repeats=num_repeats,
                                solver=dw_sampler,
                                solver_limit=subqubo_size)

    logger.removeHandler(fh)

    with open(log_file) as f:
        log_text = f.read()

    match = re.findall('solver_callback invoked', log_text)
    num_qpu_access = len(match)
    logger.info(f'Number of QPU access: {num_qpu_access}')